In [ ]:
import warnings
warnings.simplefilter("ignore", UserWarning)

In [ ]:
import pandas as pd
import numpy as np
import json
import spacy
import matplotlib.pyplot as plt
%matplotlib inline

# For regular expressions
import re

# For handling strings
import string

# For performing mathematical operations
import math

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, auc, roc_auc_score, roc_curve, f1_score, confusion_matrix
from keras import backend as K

# This module will be for saving the trained model for later use
import joblib

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model
from keras.models import Sequential

from keras.layers import Input, Dense, Embedding, Conv1D, Conv2D, MaxPooling1D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.layers import SpatialDropout1D, concatenate
from keras.layers import LSTM, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D

from keras.callbacks import Callback
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.utils.vis_utils import plot_model

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
df = pd.read_csv('cleaned_lemmatized_text.csv')
df.head()

,is_adverse_media,lemmatized_articles
0,0,zimbabweans wake news agriculture minister per...
1,1,singapore founder singapore oil trade company ...
2,1,fraudster offer green tax efficient investment...
3,1,buenos aire reuter judicial probe possible cor...
4,0,ukraines constitutional court appear strike bl...


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(df['lemmatized_articles'], 
                                                    df['is_adverse_media'], 
                                                    test_size=0.1, 
                                                    random_state=42,
                                                    stratify=df['is_adverse_media'])

print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)

(656,) (73,) (656,) (73,)


In [ ]:
MAX_NB_WORDS = 80000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, oov_token=oov_token)
tokenizer.fit_on_texts(x_train)

word_index = tokenizer.word_index

# Encode training data sentences into sequences
train_sequences = tokenizer.texts_to_sequences(x_train)

# Get max training sequence length
maxlen = 600

# Pad the training sequences
train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

# Output the results of our work
#print("Word index:\n", word_index)
#print("\nTraining sequences:\n", train_sequences)
print("\nPadded training sequences:\n", train_padded)
print("\nPadded training shape:", train_padded.shape)
print("Training sequences data type:", type(train_sequences))
print("Padded Training sequences data type:", type(train_padded))


Padded training sequences:
 [[ 197  342 3000 ... 1629  190 5034]
 [2733  164  141 ...    0    0    0]
 [9229 2865  281 ...    0    0    0]
 ...
 [ 701   42  475 ...    0    0    0]
 [5330 3876  556 ...    0    0    0]
 [ 197 1244 2584 ...    0    0    0]]

Padded training shape: (656, 600)
Training sequences data type: <class 'list'>
Padded Training sequences data type: <class 'numpy.ndarray'>


In [ ]:
val_sequences = tokenizer.texts_to_sequences(x_val)
val_padded = pad_sequences(val_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

#print("Val sequences:\n", val_sequences)
#print("\nPadded val sequences:\n", val_padded)
print("\nPadded val shape:",val_padded.shape)


Padded val shape: (73, 600)


In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def get_simple_rnn_model():
    embedding_dim = 300
    embedding_matrix = np.random.random((MAX_NB_WORDS, embedding_dim))
    
    inp = Input(shape=(maxlen, ))
    x = Embedding(input_dim=MAX_NB_WORDS, output_dim=embedding_dim, input_length=maxlen, 
                  weights=[embedding_matrix], trainable=True)(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(LSTM(100, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(1, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy', f1_m])
    return model

rnn_simple_model = get_simple_rnn_model()

In [ ]:
filepath="weights-improvement-{epoch:02d}-{val_f1_m:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_f1_m', verbose=1, save_best_only=True, mode='max')

batch_size = 128
epochs = 40

history = rnn_simple_model.fit(x=train_padded, 
                    y=y_train, 
                    validation_data=(val_padded, y_val), 
                    batch_size=batch_size, 
                    callbacks=[checkpoint], 
                    epochs=epochs, 
                    verbose=1)

Epoch 1/40
6/6 [==============================] - ETA: 0s - loss: 0.7911 - accuracy: 0.5259 - f1_m: 0.4687
Epoch 00001: val_f1_m improved from -inf to 0.00000, saving model to weights-improvement-01-0.0000.hdf5
6/6 [==============================] - 3s 552ms/step - loss: 0.7911 - accuracy: 0.5259 - f1_m: 0.4687 - val_loss: 0.7998 - val_accuracy: 0.4384 - val_f1_m: 0.0000e+00
Epoch 2/40
6/6 [==============================] - ETA: 0s - loss: 0.7191 - accuracy: 0.4924 - f1_m: 0.4248
Epoch 00002: val_f1_m improved from 0.00000 to 0.71930, saving model to weights-improvement-02-0.7193.hdf5
6/6 [==============================] - 3s 443ms/step - loss: 0.7191 - accuracy: 0.4924 - f1_m: 0.4248 - val_loss: 0.7439 - val_accuracy: 0.5616 - val_f1_m: 0.7193
Epoch 3/40
6/6 [==============================] - ETA: 0s - loss: 0.7125 - accuracy: 0.5640 - f1_m: 0.7035
Epoch 00003: val_f1_m improved from 0.71930 to 0.74074, saving model to weights-improvement-03-0.7407.hdf5
6/6 [==========================

In [ ]:
tokenizer_final = Tokenizer(num_words=MAX_NB_WORDS, oov_token=oov_token)
tokenizer_final.fit_on_texts(df['lemmatized_articles'].values)

X_train_sequences = tokenizer_final.texts_to_sequences(df['lemmatized_articles'].values)

X_train_padded = pad_sequences(X_train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
y_train_final = df['is_adverse_media'].values

In [ ]:
dependencies = {'f1_m': f1_m}
best_rnn_simple_model = load_model('weights-improvement-31-0.8916.hdf5', custom_objects=dependencies)

best_rnn_simple_model.fit(x=X_train_padded, 
                          y=y_train_final)

Epoch 1/40
6/6 [==============================] - 2s 283ms/step - loss: 0.6386 - accuracy: 0.6392 - f1_m: 0.6965
Epoch 2/40
6/6 [==============================] - 2s 278ms/step - loss: 0.5874 - accuracy: 0.6900 - f1_m: 0.7378
Epoch 3/40
6/6 [==============================] - 2s 297ms/step - loss: 0.5151 - accuracy: 0.7449 - f1_m: 0.7847
Epoch 4/40
6/6 [==============================] - 2s 283ms/step - loss: 0.4700 - accuracy: 0.8066 - f1_m: 0.8242
Epoch 5/40
6/6 [==============================] - 2s 278ms/step - loss: 0.4342 - accuracy: 0.8326 - f1_m: 0.8628
Epoch 6/40
6/6 [==============================] - 2s 283ms/step - loss: 0.4019 - accuracy: 0.8491 - f1_m: 0.8693
Epoch 7/40
6/6 [==============================] - 2s 287ms/step - loss: 0.3756 - accuracy: 0.8669 - f1_m: 0.8822
Epoch 8/40
6/6 [==============================] - 2s 282ms/step - loss: 0.3469 - accuracy: 0.8820 - f1_m: 0.8993
Epoch 9/40
6/6 [==============================] - 2s 281ms/step - loss: 0.3370 - accuracy: 0.879

In [ ]:
df_test = pd.read_csv('public_test.csv')

In [ ]:
X_test_sequences = tokenizer_final.texts_to_sequences(df_test['article'].values)

X_test_padded = pad_sequences(X_test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
y_test = df_test['label'].values


y_test_pred = best_rnn_simple_model.predict(X_test_padded, verbose=1)
y_test_pred = y_test_pred.reshape(y_test_pred.shape[0],)
y_test_pred = np.array([1 if p>=0.5 else 0 for p in y_test_pred])

5/5 [==============================] - 0s 19ms/step


In [ ]:
auc_without_glove = roc_auc_score(y_test, y_test_pred)
accuracy_without_glove = accuracy_score(y_test, y_test_pred)
f1_without_glove  = f1_score(y_test, y_test_pred)

In [ ]:
print(auc_without_glove, accuracy_without_glove, f1_without_glove)

0.8014632524110409 0.7861635220125787 0.8068181818181818
